In [ ]:
import pandas as pd
import os

In [ ]:
def is_bad_stock(stock, sp500):
    if (stock.loc[stock['Year'] == 2023, '1YRCAGR'].values[0] < sp500.loc[sp500['Year'] == 2023, '1YRCAGR'].values[0] and
        stock.loc[stock['Year'] == 2023, '5YRCAGR'].values[0] < sp500.loc[sp500['Year'] == 2023, '5YRCAGR'].values[0] and
        stock.loc[stock['Year'] == 2023, '10YRCAGR'].values[0] < sp500.loc[sp500['Year'] == 2023, '10YRCAGR'].values[0] and
        stock.loc[stock['Year'] == 2023, '15YRCAGR'].values[0] < sp500.loc[sp500['Year'] == 2023, '15YRCAGR'].values[0] and
        stock.loc[stock['Year'] == 2023, '20YRCAGR'].values[0] < sp500.loc[sp500['Year'] == 2023, '20YRCAGR'].values[0] and
        stock.loc[stock['Year'] == 2023, '25YRCAGR'].values[0] < sp500.loc[sp500['Year'] == 2023, '25YRCAGR'].values[0] and
        stock.loc[stock['Year'] == 2023, '30YRCAGR'].values[0] < sp500.loc[sp500['Year'] == 2023, '30YRCAGR'].values[0]):
        return True
    else:
        return False

In [ ]:
#as we are trying to outperform the S&P 500, create a dataframe for reference
sp500 = pd.read_csv('CAGR/SPX.csv')
#set the CAGR data directory and some dictionaries which will be the CAGR dataframe value and Ticker key
folderpath = "CAGR"
stocks = {}
bad_stocks = {}
potential_stocks = {}
#for some reason there is an issue with one of the NVR values converting. Must fix 1993 row manually.
nvr = pd.read_csv('CAGR/NVR.csv')
nvr.loc[nvr['Year'] == 1993, '1YRCAGR'] = 1850.00
for filename in os.listdir(folderpath):
    if filename.endswith(".csv"):
        stock = pd.read_csv(os.path.join(folderpath, filename))
        ticker = filename.split(".")[0]
        stocks[ticker] = stock
        if is_bad_stock(stock, sp500):
            bad_stocks[ticker] = stock
        else:
            potential_stocks[ticker] = stock
#bad stock tickers
print(len(bad_stocks),'bad stocks',bad_stocks.keys())
#potential stock tickers
print(len(potential_stocks),'potential stocks',potential_stocks.keys())

In [ ]:
def find_greater_occurences(sp500, stock, yrscagr):
    #Get the rows where yrscagr is not null
    sp500_yrs = sp500[sp500[yrscagr].notna()]
    stock_yrs = stock[stock[yrscagr].notna()]

    # Find the common years between 'sp500' and 'stock'
    common_years = set(sp500_yrs['Year']) & set(stock_yrs['Year'])

    # Initialize a counter for the number of times 'sp500' has a greater value
    sp500_greater = 0
    stock_greater = 0

    # Iterate over the common years and compare values in 'YourColumn'
    for year in common_years:
        sp500_year = sp500_yrs[sp500_yrs['Year'] == year]
        stock_year = stock_yrs[stock_yrs['Year'] == year]

        # Compare the values in 'YourColumn' for the current year
        if (sp500_year[yrscagr].values > stock_year[yrscagr].values).all():
            sp500_greater += 1
        elif (stock_year[yrscagr].values > sp500_year[yrscagr].values).all():
            stock_greater += 1
    #print(stock_greater, sp500_greater)
    return(sp500_greater, stock_greater)


In [ ]:
#create a dataframe to hold the matchup results
return_comparisons = pd.DataFrame(columns=['Ticker', '30YRWins', '30YRLosses', '25YRWins', '25YRLosses', '20YRWins', '20YRLosses', '15YRWins', '15YRLosses', '10YRWins', '10YRLosses', '5YRWins', '5YRLosses'])

In [ ]:
#print out the 30YRCAGR for aapl and sp500 for all common years. Convert to a list and print out the list
year_ranges = ['30YRCAGR', '25YRCAGR', '20YRCAGR', '15YRCAGR', '10YRCAGR', '5YRCAGR']
#for every potential stock, find the number of times it beats the sp500 in each year range
for stock in potential_stocks:
    return_comparison = [stock]
    for year_range in year_ranges:
        sp500_greater, stock_greater = find_greater_occurences(sp500, potential_stocks[stock], year_range)
        return_comparison.append(stock_greater)
        return_comparison.append(sp500_greater)
        #print(f'sp500 wins {sp500_greater} times and {stock} wins {stock_greater} times in {year_range}CAGR')
    return_comparisons.loc[len(return_comparisons)] = return_comparison
len(return_comparisons)

In [ ]:
return_comparisons.to_csv('results/return_comparisons.csv')

In [ ]:
#get the stocks that have more wins than losses against the sp500 in every size CAGR window
winners30YR = return_comparisons[return_comparisons['30YRWins'] > return_comparisons['30YRLosses']]
winners25YR = winners30YR[winners30YR['25YRWins'] > winners30YR['25YRLosses']]
winners20YR = winners25YR[winners25YR['20YRWins'] > winners25YR['20YRLosses']]
winners15YR = winners20YR[winners20YR['15YRWins'] > winners20YR['15YRLosses']]
winners10YR = winners15YR[winners15YR['10YRWins'] > winners15YR['10YRLosses']]
super_winners = winners10YR[winners10YR['5YRWins'] > winners10YR['5YRLosses']]
print(len(super_winners),'winners5YR',super_winners['Ticker'].values)

In [ ]:
super_winners.to_csv('results/super_winners.csv')

In [ ]:
super_winners_stocks = {}
for ticker in super_winners['Ticker'].values:
    super_winners_stocks[ticker] = potential_stocks[ticker]
print(len(bad_stocks),'bad stocks',bad_stocks.keys())
print(len(potential_stocks),'potential stocks',potential_stocks.keys())
print(len(super_winners_stocks),'super winners',super_winners_stocks.keys())

In [ ]:
median_cagrs = pd.DataFrame(columns=['Ticker', '30YR_median', '25YR_median', '20YR_median', '15YR_median', '10YR_median', '5YR_median'])

In [ ]:
#find the median CAGR for each year range for each stock
for stock in super_winners_stocks:
    median_cagr = [stock]
    for year_range in year_ranges:
        median_cagr.append(super_winners_stocks[stock][year_range].median())
    median_cagrs.loc[len(median_cagrs)] = median_cagr
#reorder the rows by 30YR_median
median_cagrs = median_cagrs.sort_values(by=['30YR_median'], ascending=False)
#median_cagrs.to_csv('median_cagrs.csv')

In [ ]:
#find the median CAGR for each year range for SPX and add it to the median_cagrs dataframe
sp500_median_cagr = ['SPX']
for year_range in year_ranges:
    sp500_median_cagr.append(sp500[year_range].median())
median_cagrs.loc[len(median_cagrs)] = sp500_median_cagr
median_cagrs.to_csv('results/median_cagrs.csv')